# Colab Pro benefits

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec 17 01:15:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# Importer les données WMT14

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r /content/drive/MyDrive/Projet2_NLP/data /content/

# Joey NMT Demo - Guide
Source: https://github.com/joeynmt/joeynmt/blob/master/joey_demo.ipynb

Nous allons entraîner un modèle de Transformer NMT de base de l'anglais vers le français. 

**Important:** Avant de commencer, il est indispensable de choisir le type d'exécution de GPU. 



## Installation

Installez la bonne version de PyTorch pour Joey NMT. Il faudra peut-être redémarrer le colab après avoir installé Joey NMT.

In [5]:
!pip install torch==1.9.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 831.4 MB 2.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0+cu102 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0+cu102 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9.0+cu102 which is incompatible.


In [6]:
!pip install joeynmt

     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 7.1 MB 11.7 MB/s 
     |████████████████████████████████| 90 kB 11.3 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
     |████████████████████████████████| 15.3 MB 60.0 MB/s 
     |████████████████████████████████| 735.5 MB 15 kB/s 
     |████████████████████████████████| 414 kB 70.4 MB/s 
     |████████████████████████████████| 250 kB 63.8 MB/s 
     |████████████████████████████████| 103 kB 70.9 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 843 kB 63.7 MB/s 
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68433 sha256=3fa68c0bed9ba1c05d9a02ebb215aecb4f8a2466a27314c5fc753402a5f9958b
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d/da8bad4545585ca52311498ff677647c95c7b690b3040171f8
Successfully built wrapt
  Attempting uninstall: six
    Found existing installation: six 1.15.0
  

## Continuing with the extracted data..

### Word count

In [1]:
!wc -l /content/data/en-fr/*

     3000 /content/data/en-fr/dev.en
     3000 /content/data/en-fr/dev.fr
     3003 /content/data/en-fr/test.en
     3003 /content/data/en-fr/test.fr
  2007723 /content/data/en-fr/train_full.en
  2007723 /content/data/en-fr/train_full.fr
  4027452 total


Nous n'utiliserons qu'un sous-ensemble de données pour développement et test.

In [2]:
!head -2007723  /content/data/en-fr/train_full.en > /content/data/en-fr/train.en
!head -2007723  /content/data/en-fr/train_full.fr > /content/data/en-fr/train.fr

Let's have a look at our data

In [3]:
!head -2 data/en-fr/train.* data/en-fr/dev.* data/en-fr/test.*

==> data/en-fr/train.en <==
Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.

==> data/en-fr/train.fr <==
Reprise de la session
Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.

==> data/en-fr/dev.en <==
A Republican strategy to counter the re-election of Obama 
Republican leaders justified their policy by the need to combat electoral fraud.

==> data/en-fr/dev.fr <==
Une stratégie républicaine pour contrer la réélection d'Obama
Les dirigeants républicains justifièrent leur politique par la nécessité de lutter contre la fraude électorale.

==> data/en-fr/test.en <==
Jet makers feud over seat width with big orders at stake
A row has flared up between leading p

## Entraînement d'un modèle de Subword

Nous utiliserons la bibliothèque `subword_nmt` pour diviser les mots en sous-mots (BPE) en fonction de leur fréquence dans le corpus d'apprentissage.



In [4]:
import os

In [5]:
src_lang = 'en'
trg_lang = 'fr'
bpe_size = 20000    # taille du vocabulaire à définir
datadir = '/content/data/en-fr/'
name = f'{src_lang}_{trg_lang}_bpe{bpe_size}'


train_src_file = os.path.join(datadir, f'train.{src_lang}')
train_trg_file = os.path.join(datadir, f'train.{trg_lang}')
train_joint_file = os.path.join(datadir, f'train.{src_lang}-{trg_lang}')
dev_src_file = os.path.join(datadir, f'dev.{src_lang}')
dev_trg_file = os.path.join(datadir, f'dev.{trg_lang}')
test_src_file = os.path.join(datadir, f'test.{src_lang}')
test_trg_file = os.path.join(datadir, f'test.{trg_lang}')
src_files = {'train': train_src_file, 'dev': dev_src_file, 'test': test_src_file}
trg_files = {'train': train_trg_file, 'dev': dev_trg_file, 'test': test_trg_file}


vocab_src_file = os.path.join(datadir, f'vocab.{bpe_size}.{src_lang}')
vocab_trg_file = os.path.join(datadir, f'vocab.{bpe_size}.{trg_lang}')
bpe_file = os.path.join(datadir, f'bpe.codes.{bpe_size}')

Entraîner un modèle BPE avec 20000 symboles pour les deux langues conjointement.



In [6]:
! cat $train_src_file $train_trg_file > $train_joint_file

! subword-nmt learn-bpe \
  --input $train_joint_file \
  -s $bpe_size \
  -o $bpe_file

100% 20000/20000 [01:46<00:00, 187.41it/s]


Ce fichier contient les fusions de séquences de caractères qui composent les sous-mots.

In [7]:
! head $bpe_file

#version: 0.2
e n
o n
t i
t h
e s</w>
a n
o u
i n
r e


Nous appliquons les fusions BPE apprises aux données de formation, de développement et de test.


In [8]:
src_bpe_files = {}
trg_bpe_files = {}
for split in ['train', 'dev', 'test']:
  src_input_file = src_files[split]
  trg_input_file = trg_files[split]
  src_output_file = src_input_file.replace(split, f'{split}.{bpe_size}.bpe')
  trg_output_file = trg_input_file.replace(split, f'{split}.{bpe_size}.bpe')
  src_bpe_files[split] = src_output_file
  trg_bpe_files[split] = trg_output_file

  ! subword-nmt apply-bpe \
    -c $bpe_file \
    < $src_input_file > $src_output_file

  ! subword-nmt apply-bpe \
    -c $bpe_file \
    < $trg_input_file > $trg_output_file


Les données subword-split contiennent `@@ ` pour indiquer où les mots ont été divisés en sous-mots.

In [9]:
! head data/en-fr/dev.20000.bpe.en

A Re@@ publi@@ can strategy to counter the re-@@ election of Ob@@ ama 
Re@@ publi@@ can leaders justified their policy by the need to combat electoral frau@@ d.
However, the B@@ ren@@ n@@ an Centre considers this a my@@ th, stating that electoral fraud is ra@@ rer in the United States than the number of people killed by ligh@@ t@@ ning.
Indeed, Re@@ publi@@ can law@@ yers identified only 300 cases of electoral fraud in the United States in a deca@@ de.
One thing is certain@@ : these new provisions will have a negative impact on voter tur@@ n-@@ out.
In this sense, the measures will parti@@ ally undermine the American democratic system.
Un@@ like in Canada, the American States are responsible for the organisation of federal elections in the United States.
It is in this spirit that a majority of American governments have passed new laws since 2009 making the registration or voting process more difficult.
This phenomenon gained moment@@ um following the November 2010 elections, which saw 

In [10]:
! head data/en-fr/dev.20000.bpe.fr

Une stratégie ré@@ public@@ aine pour contrer la ré@@ élection d'@@ Ob@@ ama
Les dirigeants ré@@ public@@ ains justifi@@ èrent leur politique par la nécessité de lutter contre la fraude élector@@ ale.
Or, le Centre B@@ ren@@ n@@ an considère cette dernière comme un my@@ th@@ e, affirm@@ ant que la fraude électorale est plus rare aux États-Unis que le nombre de personnes tu@@ ées par la fou@@ d@@ re.
D'@@ ailleurs, les avoc@@ ats ré@@ public@@ ains n'ont re@@ cen@@ sé que 300 cas de fraude électorale aux États-Unis en dix ans.
Une chose est certain@@ e: ces nouvelles dispositions influ@@ en@@ cer@@ ont nég@@ ati@@ vement le taux de particip@@ ation.
En ce sens, ces mesures min@@ eront en partie le système démocratique américain@@ .
Contr@@ airement au Canada, les États américains sont responsables de l'organisation des élections fédér@@ ales aux États-Unis.
C'est dans cet esprit qu'une majorité de gouvernements américains prom@@ ul@@ gu@@ èrent à partir de 2009 de nouvelles lois rendant

## Préparer le vocabulaire

À partir des données d'entraînement prétraitées, nous extrayons le vocabulaire final du modèle de traduction. Il doit contenir tous les sous-mots nécessaires pour représenter les données d'entraînement source et cible.

In [11]:
! wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py

--2021-12-17 01:26:46--  https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2034 (2.0K) [text/plain]
Saving to: ‘build_vocab.py’

build_vocab.py      100%[===================>]   1.99K  --.-KB/s    in 0s      

2021-12-17 01:26:47 (39.7 MB/s) - ‘build_vocab.py’ saved [2034/2034]



In [12]:
vocab_src_file = src_bpe_files['train']
vocab_trg_file = trg_bpe_files['train']
bpe_vocab_file = os.path.join(datadir, f'joint.{bpe_size}bpe.vocab')

! python build_vocab.py  \
  $vocab_src_file $vocab_trg_file \
  --output_path $bpe_vocab_file

# Configuration du modèle

Joey NMT lit les hyperparamètres de modèle et d'entraînement à partir d'un fichier de configuration. Nous générons ceci maintenant pour configurer les chemins aux endroits appropriés.

La configuration ci-dessous crée un petit modèle Transformer avec des intégrations partagées entre la langue source et la langue cible sur la base des vocabulaires de sous-mots créés ci-dessus.

In [13]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{datadir}/train.{bpe_size}.bpe"
    dev:   "{datadir}/dev.{bpe_size}.bpe"
    test:  "{datadir}/test.{bpe_size}.bpe"
    level: "bpe"
    lowercase: False                
    max_sent_length: 100             # Extend to longer sentences. **
    src_vocab: "{vocab_src_file}"
    trg_vocab: "{vocab_trg_file}"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "none"            # `tokenize` option in sacrebleu.corpus_bleu() function (options include: "none" (use for already tokenized test data), "13a" (default minimal tokenizer), "intl" which mostly does punctuation and unicode, etc) 

training:
    #load_model: "models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # Alternative: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    #eval_batch_size: 3600
    #eval_batch_type: "token"
    #batch_multiplier: 1
    early_stopping_metric: "loss"
    epochs: 20                     # Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2000          # Set to at least once per epoch.
    logging_freq: 200
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3    

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True       # Requires joint vocabulary.
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8             # Increase 4 to 8 for larger data.
        embeddings:
            embedding_dim: 512   # Increase 256 to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # Increase 256 to 512 for larger data.
        ff_size: 2048            # Increase 1024 to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # Increase 4 to 8 for larger data.
        embeddings:
            embedding_dim: 512    # Increase 256 to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase 256 to 512 for larger data.
        ff_size: 2048            # TODO: Increase 1024 to 2048 for larger data.
        dropout: 0.3
""".format(name=name, source_language=src_lang, target_language=trg_lang,
           datadir=datadir, vocab_src_file=bpe_vocab_file, 
           vocab_trg_file=bpe_vocab_file, bpe_size=bpe_size)
with open("transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Entraînement

Cela va prendre du temps. Le journal rapporte le processus de formation, recherchez les impressions d'exemples de traductions et les notes d'évaluation BLEU pour avoir une idée de la qualité actuelle.

Le journal est également stocké dans le répertoire du modèle au sein de ce runtime (inspectez les fichiers dans le menu de gauche). Vous y trouverez également un rapport récapitulatif de toutes les validations. Nous utiliserons également TensorBoard pour visualiser la progression de l'entraînement lors de vos déplacements. Cela nécessite l'activation des cookies dans le navigateur.

Après 12h au plus tard, Colab se déconnectera, donc pour vous assurer que votre progression n'est pas perdue, téléchargez les points de contrôle depuis le répertoire des modèles de temps en temps. Vous pourrez plus tard les recharger si les hyperparamètres du modèle correspondent.

In [14]:
! pip install --upgrade sacrebleu==1.5.1
# conflit de version sacreBleu==2.0 => donc réinstaller l'ancienne version

     |████████████████████████████████| 54 kB 3.1 MB/s 
  Attempting uninstall: portalocker
    Found existing installation: portalocker 2.3.2
    Uninstalling portalocker-2.3.2:
      Successfully uninstalled portalocker-2.3.2
  Attempting uninstall: sacrebleu
    Found existing installation: sacrebleu 2.0.0
    Uninstalling sacrebleu-2.0.0:
      Successfully uninstalled sacrebleu-2.0.0


In [33]:
# Chargez l'extension de bloc-notes TensorBoard. Il sera vide au début.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [32]:
!kill 529

/bin/bash: line 0: kill: (529) - No such process


In [34]:
%tensorboard --logdir models/en_fr_bpe20000_transformer/tensorboard 

Reusing TensorBoard on port 6006 (pid 529), started 8:20:25 ago. (Use '!kill 529' to kill it.)

<IPython.core.display.Javascript object>

In [18]:
!python -m joeynmt train transformer_en_fr_bpe20000.yaml

2021-12-17 01:28:59,016 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-12-17 01:28:59,065 - INFO - joeynmt.data - Loading training data...
2021-12-17 01:29:42,197 - INFO - joeynmt.data - Building vocabulary...
2021-12-17 01:29:48,862 - INFO - joeynmt.data - Loading dev data...
2021-12-17 01:29:49,075 - INFO - joeynmt.data - Loading test data...
2021-12-17 01:29:49,112 - INFO - joeynmt.data - Data loaded.
2021-12-17 01:29:49,112 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-17 01:29:50,068 - INFO - joeynmt.model - Enc-dec model built.
2021-12-17 01:29:54,833 - INFO - joeynmt.training - Total params: 54624256
2021-12-17 01:30:00,333 - INFO - joeynmt.helpers - cfg.name                           : en_fr_bpe20000_transformer
2021-12-17 01:30:00,336 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-12-17 01:30:00,336 - INFO - joeynmt.helpers - cfg.data.trg                       : fr
2021-12-17 01:30:00,336 - INFO - joeynmt.helpers - 

## Continuer l'entraînement après interruption

Pour continuer après une interruption, la configuration doit être modifiée à 2 endroits :
1. `load_model` pour pointer vers le point de contrôle à charger.
2. `model_dir` pour créer un nouveau répertoire.


In [27]:
ckpt_number = 2000
reload_config = config.replace(
    f'#load_model: "models/{name}_transformer/1.ckpt"', f'load_model: "models/{name}_transformer/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/{name}_transformer"', f'model_dir: "models/{name}_transformer_continued"')
with open("transformer_{name}_reload.yaml".format(name=name),'w') as f:
    f.write(reload_config)

Joey NMT reprend alors l'entraînement à partir de là.



In [28]:
!python -m joeynmt train transformer_epo_eng_bpe20000_reload.yaml

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/joeynmt/__main__.py", line 48, in <module>
    main()
  File "/usr/local/lib/python3.7/dist-packages/joeynmt/__main__.py", line 35, in main
    train(cfg_file=args.config_path, skip_test=args.skip_test)
  File "/usr/local/lib/python3.7/dist-packages/joeynmt/training.py", line 761, in train
    cfg = load_config(cfg_file)
  File "/usr/local/lib/python3.7/dist-packages/joeynmt/helpers.py", line 178, in load_config
    with open(path, 'r') as ymlfile:
FileNotFoundError: [Errno 2] No such file or directory: 'transformer_epo_eng_bpe20000_reload.yaml'


Sinon, supprimer l'ancien dossier models, puis réfaire l'entraînement. 

In [ ]:
# !rm -fr models  # pour supprimer un dossier de models , par exemple pour pouvoir ré-entraîner un autre modèle

# Évaluation et Prédiction

Le mode `test` peut être utilisé pour traduire (et évaluer sur) l'ensemble de test spécifié dans la configuration. Nous ne le faisons généralement qu'une seule fois après avoir réglé les hyperparamètres sur l'ensemble de développement.

In [19]:
!python -m joeynmt test models/en_fr_bpe20000_transformer/config.yaml

# pour 10 itérations sur (train, dev, test) = (181k, 1k, 1k)
# dev bleu[none]:   0.32 
# test bleu[none]:   0.25

2021-12-17 09:14:17,908 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-12-17 09:14:17,909 - INFO - joeynmt.data - Building vocabulary...
2021-12-17 09:14:23,886 - INFO - joeynmt.data - Loading dev data...
2021-12-17 09:14:23,922 - INFO - joeynmt.data - Loading test data...
2021-12-17 09:14:23,962 - INFO - joeynmt.data - Data loaded.
2021-12-17 09:14:24,008 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 20480 (with beam_size)
2021-12-17 09:14:29,712 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-17 09:14:30,625 - INFO - joeynmt.model - Enc-dec model built.
2021-12-17 09:14:30,853 - INFO - joeynmt.prediction - Decoding on dev set (/content/data/en-fr//dev.20000.bpe.fr)...
2021-12-17 09:17:17,436 - INFO - joeynmt.prediction -  dev bleu[none]:  19.80 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-12-17 09:17:17,437 - INFO - joeynmt.prediction - Decoding on test set (/content/data/en-fr//test.20000.bpe



Le mode `translate` est plus interactif et prend des invites pour traduire de manière interactive. Attention : cela nécessite d'appliquer les mêmes étapes de pré-traitement à la nouvelle entrée que celles que vous avez appliquées avant l'entraînement du modèle (c'est-à-dire la segmentation en sous-mots).

In [24]:
from subword_nmt import apply_bpe

with open(bpe_file, "r") as merge_file:
  bpe = apply_bpe.BPE(codes=merge_file)

preprocess = lambda x: bpe.process_line(x.strip())

In [21]:
my_sentence = "I don't know whether machine translation will eventually get good enough to allow us to browse people's websites in different languages so you can see how they live in different countries."
#my_sentence = 'Please enter a source sentence.'  

# référence : Toutes les parties prenantes auraient intérêt à lancer un autre moteur.

In [22]:
preprocess(my_sentence)

"I don@@ '@@ t know whether mach@@ ine transl@@ ation will eventually get good enough to allow us to bro@@ w@@ se people's web@@ sites in different languages so you can see how they live in different countries."

Copiez la phrase prétraitée ci-dessus dans le champ lorsque vous y êtes invité ci-dessous. Arrêtez la cellule pour quitter le mode interactif.

In [23]:
!python -m joeynmt translate models/en_fr_bpe20000_transformer/config.yaml

2021-12-17 09:20:35,125 - INFO - root - Hello! This is Joey-NMT (version 1.3).
I don@@ '@@ t know whether mach@@ ine transl@@ ation will eventually get good enough to allow us to bro@@ w@@ se people's web@@ sites in different languages so you can see how they live in different countries.
2021-12-17 09:20:45,930 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-17 09:20:46,771 - INFO - joeynmt.model - Enc-dec model built.

Please enter a source sentence (pre-processed): 
JoeyNMT: Hypotheses ranked by score
JoeyNMT #1: Je ne sais pas si la traduction de la machine nous permettra de briser les sites web des citoyens dans différentes langues. Vous pouvez donc voir comment ils vivent dans différents pays.

Please enter a source sentence (pre-processed): 

Bye.Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joeynmt/prediction.py", line 505, in translate
    src_input = input("\nPlease enter a source sentence "
KeyboardInterrupt

During ha

Vous pouvez également obtenir les n meilleures hypothèses (jusqu'à la taille du faisceau, dans notre exemple 5), pas seulement la plus élevée. Plus votre modèle s'améliore, plus les alternatives devraient être intéressantes.


In [ ]:
!python -m joeynmt translate models/en_fr_bpe20000_transformer/config.yaml -n 5

2021-12-16 02:33:37,254 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-12-16 02:34:16,521 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-16 02:34:17,672 - INFO - joeynmt.model - Enc-dec model built.

Please enter a source sentence (pre-processed): 
It would be benefit all parties to start another motor@@
JoeyNMT: Hypotheses ranked by score
JoeyNMT #1: Il serait bénéfique que toutes les parties commencent à une autre autoroute.
JoeyNMT #2: Il serait bénéfique que toutes les parties commencent à lancer une autre autoroute.
JoeyNMT #3: Il serait bon que toutes les parties commencent à lancer une autre autoroute.
JoeyNMT #4: Il serait utile que toutes les parties commencent à lancer une autre autoroute.
JoeyNMT #5: Il serait bénéfique que toutes les parties commencent à un autre autoroute.

Please enter a source sentence (pre-processed): 

Bye.Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joeynmt/prediction.py", line 5

## Récupérer le test de prédiction

In [ ]:
!python -m joeynmt test models/en_fr_bpe20000_transformer/config.yaml --output_path data/en-fr/out

2021-12-16 02:34:56,923 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-12-16 02:34:56,924 - INFO - joeynmt.data - Building vocabulary...
2021-12-16 02:35:32,991 - INFO - joeynmt.data - Loading dev data...
2021-12-16 02:35:33,025 - INFO - joeynmt.data - Loading test data...
2021-12-16 02:35:33,059 - INFO - joeynmt.data - Data loaded.
2021-12-16 02:35:33,075 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 20480 (with beam_size)
2021-12-16 02:35:36,216 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-16 02:35:37,374 - INFO - joeynmt.model - Enc-dec model built.
2021-12-16 02:35:37,660 - INFO - joeynmt.prediction - Decoding on dev set (/content/data/en-fr//dev.50000.bpe.fr)...
2021-12-16 02:37:30,926 - INFO - joeynmt.prediction -  dev bleu[none]:  16.34 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-12-16 02:37:30,928 - INFO - joeynmt.prediction - Translations saved to: data/en-fr/out.dev
2021-12-16 02:3

In [ ]:
!cp -r models/ /content/drive/MyDrive/Projet2_NLP

Fin du guide !
